# Conifer Synthesis with GradientBoost BDT on ATLAS Tau Data

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics
import conifer
import datetime
import numpy as np
import matplotlib.pyplot as plt
import itertools

### Load GradientBoostingClassifier

In [ ]:
with open('GradientBoosted_params5_trees50_depth2.pkl', 'rb') as f:
    clf = pickle.load(f)

### Determine model accuracy

In [ ]:
from sklearn.metrics import accuracy_score
y_predict = clf.predict(X_test)
print("Accuracy: {}".format(accuracy_score(y_test, y_predict)))

### Determine AUCs and ROC curves

In [ ]:
y_score = clf.decision_function(X_test)
fpr = dict()
tpr = dict()
roc_auc = dict()
y_test_bin = sklearn.preprocessing.label_binarize(y_test, classes=le.transform(list(le.classes_)))
for i in range(y_test_bin.shape[1]):
    fpr[i], tpr[i], _ = sklearn.metrics.roc_curve(y_test_bin[:,i], y_score[:,i])
    roc_auc[i] = sklearn.metrics.auc(fpr[i], tpr[i])

### Plot ROC Curve

In [ ]:
# Plot all ROC curves
plt.figure()

lw=2
n_classes=len(list(le.classes_))

from itertools import cycle
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label= list(le.classes_)[i] + ' tagger (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for BDT Jet Classifier')
plt.legend(loc="lower right")
plt.show()

### Plot ROC Curve as Signal Efficiency vs Log of Background Efficiency

In [ ]:
# Plot all ROC curves
plt.figure()

lw=2
n_classes=len(list(le.classes_))

from itertools import cycle
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green'])
for i, color in zip(range(n_classes), colors):
    plt.plot(tpr[i], fpr[i], color=color, lw=lw,
             label= list(le.classes_)[i] + ' tagger (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.semilogy()
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('Signal Efficiency')
plt.ylabel('Background Efficiency')
plt.title('ROC Curves for BDT Jet Classifier')
plt.legend(loc="upper left")
plt.show()

### Generate confusion matrix

In [ ]:
cm =sklearn.metrics.confusion_matrix(y_test, y_predict, labels=le.transform(list(le.classes_)))

In [ ]:
# confusion matrix code from Maurizio
# /eos/user/m/mpierini/DeepLearning/ML4FPGA/jupyter/HbbTagger_Conv1D.ipynb
def plot_confusion_matrix(cm, classes,
                          normalize=False, 
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    #plt.title(title)
    cbar = plt.colorbar()
    plt.clim(0,1)
    cbar.set_label(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm,list(le.classes_), normalize=True)

### Perform model synthesis

In [ ]:
# Create a conifer config
cfg = conifer.backends.vivadohls.auto_config()
# Set the output directory to something unique
cfg['OutputDir'] = 'tauTrees/prj_{}'.format(int(datetime.datetime.now().timestamp()))

# Create and compile the model
model = conifer.model(clf, conifer.converters.sklearn, conifer.backends.vivadohls, cfg)
model.compile()

# Run HLS C Simulation and get the output
y_conif = model.decision_function(X_test)

# Synthesize the model
model.build()

### Generate Resource Reports

In [ ]:
import hls4ml
hls4ml.report.read_vivado_report(cfg['OutputDir'])

### Generate ROC curve metrics for conifer model

In [ ]:
fpr_conif = dict()
tpr_conif = dict()
roc_auc_conif = dict()
for i in range(y_test_bin.shape[1]):
    fpr_conif[i], tpr_conif[i], _ = sklearn.metrics.roc_curve(y_test_bin[:,i], y_conif[:,i])
    roc_auc_conif[i] = sklearn.metrics.auc(fpr_conif[i], tpr_conif[i])

### Plot ROC curves comparing sklearn model to conifer model

In [ ]:
# Plot all ROC curves
plt.figure(figsize=[8,8])

lw=2
n_classes=len(list(le.classes_))

from itertools import cycle
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green'])
for i, color in zip(range(n_classes), colors):
    plt.plot(tpr[i], fpr[i], color=color, lw=lw,
             label= list(le.classes_)[i] + ' tagger (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
    plt.plot(tpr_conif[i], fpr_conif[i], '--', color=color, lw=lw,
             label= list(le.classes_)[i] + ' tagger conifer (area = {1:0.2f})'
             ''.format(i, roc_auc_conif[i]))    

plt.semilogy()
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Signal Efficiency')
plt.ylabel('Background Efficiency')
plt.title('ROC Curves for BDT Jet Classifier')
plt.legend(loc="upper left")
plt.show()